# Task: Build Transformer (TF) for implementing linear_search

In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

## Here we show how we can implement linear_search() function with looped TF. 

The input/output of the linear_search function we are implementing is:
- Input: an array of scalars (denoted as input_list = [a_1, ..., a_L]), a target number T
- Output: if the array contains the target number, it outputs the index  i satisfying a_i = T; otherwise, it says the array does not have the target number

This jupyter notebook is organized as below:

1. We first specify 
- the memory & SUBLEQ commands to define the linear_search function 
- the parameters used for building the looped TF 

2. We run linear_search with SUBLEQ commands manually (without using TF)

3. We run linear_search with SUBLEQ commands by Looped TF

4. We check whether our Looped TF implementation result matches with ground truth value

## 1. Setup

### Specify the parameters for the program linear_search

In [4]:
task_name = 'linear_search'
input_list = [-7, -6, -5, -4, 3, 2, 1] # put the list we have
T = 1 # put the target number we are searching for in this list
L = len(input_list)

In [5]:
# write subleq commands & initial memory for the given input_ftrl
import csv

## memory (written in "name=initial_value" format)
# [M_1] MINUS_ONE = -1 (we set a constant for implementing EOF)
# [M_2] ZERO = 0 (we set a constant for implementing EOF)
# [M_3] a_1 = input_list[0]
# ...
# [M_{L+2}] a_L = input_list[L-1]
# [M_{L+3}] temp_1 = 0
# [M_{L+4}] temp_2 = 0
# [M_{L+5}] target = T
# [M_{L+6}] flag = 1   # save whether we failed (flag=1: failure, flag=0: success)
# [M_{L+7}] index = 0 # save the index (where the target number is located in the list)

mems = np.concatenate((np.array([-1, 0]), 
                      np.asarray(input_list), 
                      np.array([0, 0, T, 1, 0]))).reshape(-1, 1) 
mems_path = 'inputs/{}_init_mem.csv'.format(task_name)
pd.DataFrame(mems).to_csv(mems_path, header  = ['mem'], index=False)    

## commands 
# NOTE: subleq a,b,c does two things:
# 1. mem[b] = mem[b] - mem[a]
# 2. if mem[b] <= 0: goto instruction c
#   else: goto next instruction

# [C_1] subleq a_i, temp_2, C_2         (set temp_2 = -a_i)
# [C_2] subleq temp_2, temp_1, C_3      (set temp_1 = a_i)
# [C_3] subleq target, temp_1, C_6      (check whether temp_1 = a_i - T <= 0)
# [C_4] subleq temp_1, temp_1, C_5      (reset temp_1 = 0)
# [C_5] subleq temp_2, temp_2, C_11     (reset temp_2 = 0, failure)
# [C_6] subleq temp_2, temp_2, C_7      (reset temp_2 = 0)
# [C_7] subleq temp_1, temp_2, C_10.    (check whether temp_2 = T - a_i <= 0)
# [C_8] subleq temp_1, temp_1, C_9      (reset temp_1 = 0)
# [C_9] subleq temp_2, temp_2, C_11     (reset temp_2 = 0, failure)
# [C_10] subleq flag, flag, EOF    (set flag=0 and go to EOF, success)
# [C_11] subleq MINUS_ONE, index, C_12  (index+=1, go to a_2)

# ... (concatenate above for i=1, ..., L, to check whether a_i = T)

# [C_{11L}] subleq MINUS_ONE, index, C_{11L+1}
# [C_{11L+1}] subleq ZERO, MINUS_ONE, C_{11L+1} # EOF


cmds = []
EOF = 11*L+1
for l in range(L):
    cmds.append([3+l,L+4,11*l+2])   
    cmds.append([L+4,L+3,11*l+3]) 
    cmds.append([L+5,L+3,11*l+6]) 
    cmds.append([L+3,L+3,11*l+5]) 
    cmds.append([L+4,L+4,11*l+11]) 
    cmds.append([L+4,L+4,11*l+7])     
    cmds.append([L+3,L+4,11*l+10])
    cmds.append([L+3,L+3,11*l+9])
    cmds.append([L+4,L+4,11*l+11])
    cmds.append([L+6,L+6,EOF])
    cmds.append([1,L+7,11*l+12])
cmds.append([2,1,EOF])
cmds = np.asarray(cmds)
#cmds.shape

### Specify the parameters for defining the matrix X (input to TF)

In [6]:
num_cmds = cmds.shape[0]
num_mems = mems.shape[0]
s = 2 # s: number of columns in X for scratchpad (larger than 1 is enough?)
m = num_mems # m: number of columns in X for memory
n = s + m + num_cmds # n: number of columns in X in total (for scratchpad, memory and commands)
logn = int(np.ceil(np.log2(n)))

# decide N based on the memory element with the largest magnitude
max_element = np.max(np.abs(input_list)) # max_i |a_i|
max_diff = np.max(np.abs(np.asarray(input_list) - T)) # max_i |a_i - T|
max_index = L
max_mem = np.amax([max_element, max_diff, max_index])
#print('max memory: ', max_mem)
N = int(np.floor(np.log2(max_mem)))+3 # N: number of bits used to represent the integer values in each memory element


# Specify the number of rows of input X
# nrows_list: list of number of rows for each block (cmds, memory, scratchpad, program counter, positional encoding, buffer, indicator)
# row_idx_list: list of row index each block starts (memory, scratchpad, program counter, positional encoding, buffer) except cmds & indicator
from utils import get_nrows_subleq, get_row_idx_list_subleq
nrows_cmds, nrows_memory, nrows_scratchpad, nrows_pc, nrows_pos_enc, nrows_buffer = get_nrows_subleq(logn, N) 
num_rows_X = nrows_cmds + nrows_memory + nrows_scratchpad + nrows_pc + nrows_pos_enc + nrows_buffer + 1 
nrows_list = [nrows_cmds, nrows_memory, nrows_scratchpad, nrows_pc, nrows_pos_enc, nrows_buffer, 1] 
row_idx_list = get_row_idx_list_subleq(nrows_list) 
idx_memory, idx_scratchpad, idx_pc, idx_pos_enc, idx_buffer = row_idx_list

### revise & save the cmds.csv 

In [7]:
## add -1 to the address info (mem[a], mem[b], next_cmd) in SUBLEQ cmds
cmds = cmds - 1

## change the index of memory & command in cmds.txt [i -> s+i] [j -> s+m+j]
cmds[:, :2] = cmds[:, :2] + s
cmds[:, 2] = cmds[:, 2] + s + m

In [8]:
cmds_path = 'inputs/{}_cmds.csv'.format(task_name)
pd.DataFrame(cmds).to_csv(cmds_path, header  = ['a','b','c'], index=False)    

### Load input text files: (1) the subleq commands, (2) the registers

In [9]:
# load the input files 
cmds_filename = 'inputs/{}_cmds.csv'.format(task_name)
cmds_df = pd.read_csv(cmds_filename)
cmds = cmds_df.to_numpy()
mem_filename = 'inputs/{}_init_mem.csv'.format(task_name)
mem_df = pd.read_csv(mem_filename)
mem = mem_df.to_numpy().reshape(-1,)

In [10]:
# check the validity of input files
for i in range(len(cmds)):
    #print(i)
    (cmd_a, cmd_b, cmd_c) = cmds[i]
    assert(cmd_a >= s)   # a, b \in [s:s+m] 
    assert(cmd_a < s+m)   
    assert(cmd_b >= s)
    assert(cmd_b < s+m)    
    assert(cmd_c >= s+m) # c \in [s+m:n]
    assert(cmd_c < n)        
assert(len(mem) == m) # mem should have $m$ elements

for i in range(len(mem)):
    assert(mem[i] <= 2**(N-2)-1)
    assert(mem[i] >= -2**(N-2))    
    


### Define the matrix X

In [11]:
from utils import init_input
X, _ = init_input(s,m,n,logn,N,num_rows_X,cmds,nrows_list,row_idx_list,opt=None,mem_given=mem)

## 2. Run factorial (using subleq) manually

In [12]:
num_loops = 11*L+2 
from utils import run_manual_subleq
manual_subleq_results = run_manual_subleq(cmds, mem, s, m, n, N, num_loops=num_loops)

In [14]:
header = ['mem[0]']
for i in range(m-2):
    header.append('...')
header = np.append(header, ['mem[m-1]', 'a', 'b', 'c', 'mem[a]', 'mem[b]', 'mem[b]-mem[a]', 'flag', 'p-next'])
manual_subleq_results_df = pd.DataFrame(manual_subleq_results, columns=header)

## 3. Run SUBLEQ using our TF architecture

In [15]:
from subleq import read_inst, read_mem, subtract_mem, write_mem, conditional_branching, error_correction
import os
our_subleq_results = []
our_curr_result = [] 
lam=100 # lambda used for softmax in TF # need to increase as input_fctl increases?

for i in range(num_loops):
    #print('we are in loop ', i)
    
    # Step 1. read instruction & check a, b, c 
    X1, our_curr_result, TF_read_inst = read_inst(X,s,m,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 2. read memory & check mem[a], mem[b]
    X2, our_curr_result, TF_mem = read_mem(X1,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 3. subtract memory & check mem[b]-mem[a] 
    X3, our_curr_result, *TF_subtract_mem = subtract_mem(X2,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 4. write memory 
    X4, our_curr_result, TF_write_mem = write_mem(X3,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 5. conditional branching & check flag, p_{next}
    X5, our_curr_result, *TF_cond_branch = conditional_branching(X4,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 6. error correction        
    X6, our_curr_result, TF_err_corr = error_correction(X5,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 

    X = X6 # go to the next loop
    our_subleq_results.append(tuple(our_curr_result))
    #print(our_curr_result)

In [16]:
our_subleq_results_df = pd.DataFrame(our_subleq_results, columns=header)
#our_subleq_results_df

## 4. Compare our TF SUBLEQ result & manual SUBLEQ result

In [66]:
## memory (written in "name=initial_value" format)
# [M_1] MINUS_ONE = -1 (we set a constant for implementing EOF)
# [M_2] ZERO = 0 (we set a constant for implementing EOF)
# [M_3] a_1 = input_list[0]
# ...
# [M_{L+2}] a_L = input_list[L-1]
# [M_{L+3}] temp_1 = 0
# [M_{L+4}] temp_2 = 0
# [M_{L+5}] target = T
# [M_{L+6}] flag = 0   # save whether we successed
# [M_{L+7}] index = 0 # save the index (where the target number is located in the list)

In [17]:
print('original list: ', input_list)
print('target number: ', T)
print('applying {} operation'.format(task_name))

original list:  [-7, -6, -5, -4, 3, 2, 1]
target number:  1
applying linear_search operation


In [18]:
## Ground truth result
true_indices = np.where(np.array(input_list) == T)[0]
if len(true_indices) == 0:
    true_failure = 1
    true_index = None
    print("Ground Truth: The target number {} is NOT included in the list {}".format(T, input_list))
else:
    true_failure = 0
    true_index = true_indices[0]
    print("Ground Truth: The target number {} is included in the list {}, for the first time at index {}".format(T, input_list, true_index))    

Ground Truth: The target number 1 is included in the list [-7, -6, -5, -4, 3, 2, 1], for the first time at index 6


In [19]:
## Manual SUBLEQ result
manual_failure = manual_subleq_results[-1][m-2]
manual_index = manual_subleq_results[-1][m-1]
if manual_failure:
    manual_index = None
    print("Manual SUBLEQ: The target number {} is NOT included in the list {}".format(T, input_list))
else:
    print("Manual SUBLEQ: The target number {} is included in the list {}, for the first time at index {}".format(T, input_list, manual_index))            

Manual SUBLEQ: The target number 1 is included in the list [-7, -6, -5, -4, 3, 2, 1], for the first time at index 6


In [20]:
## TF SUBLEQ result
TF_failure = our_subleq_results[-1][m-2]
TF_index = our_subleq_results[-1][m-1]
if TF_failure:
    TF_index = None
    print("TF SUBLEQ: The target number {} is NOT included in the list {}".format(T, input_list))
else:
    print("TF SUBLEQ: The target number {} is included in the list {}, for the first time at index {}".format(T, input_list, TF_index))

TF SUBLEQ: The target number 1 is included in the list [-7, -6, -5, -4, 3, 2, 1], for the first time at index 6


In [21]:
if [true_index, manual_index, TF_index] is [None, None, None]:
    print('The {} in TF works properly!'.format(task_name))
elif true_index == manual_index and true_index == TF_index:
    print('The {} in TF works properly!'.format(task_name))
else:
    print('Check the code again :p')

The linear_search in TF works properly!
